In [1]:
class NaiveBayes():

    def __init__(self,k=1):        
        self.k = k
        self.X = None
        self.y = None
        self.uniqueClas = None
        self.p_c = []
        self.list_p_condics = []
        self.probabilidades_c = {}
        self.flag_entrenado = False
        
    def entrena(self,X,y):
        
        self.X = X
        self.y = y
    
        #countsClas nos da el nº de veces se da un valor de clasificación, uniqueClas los dos posibles valores de 
        #clasificación ('si' y 'no')
        uniqueClas, countsClas = np.unique(y, return_counts=True)
        self.uniqueClas = uniqueClas
    
        #Función de probabilidad P(C=c)
        p_c = []
        for i in range(len(uniqueClas)):
            p_c.append(countsClas[i]/len(y))
            
        self.p_c = p_c
        
        #Listado de probabilidades condicionales
        list_p_condics = []
        self.list_p_condics = list_p_condics
        
        for i in list(uniqueClas): #Por cada valor de clasificación ('si' o 'no'), vamos a añadir un registro en list_p_condics
            p_condics =[] #Matriz para el valor de clasificación i
            
            for j in range(X.shape[1]): #Por cada atributo, vamos a calcular sus probabilidades condicionadas. 
                                        #j es el nº de atributo 
        
                valuesAtribj, numberOfValuesAtribj = np.unique(X[:,j], return_counts=True)
                p_condics_j = [] #Por cada atributo, tenemos la fila de las probabilidades condicionadas
                
                for k in range(len(valuesAtribj)): #Por cada valor que puede tomar el atributo,
                                                   #vamos a calcular su probabilidad condicionada
                    
                    positionsOfk = np.where(X[:,j] == valuesAtribj[k]) #Posiciones en las que el atributo j vale k
                    #Formamos el vector y con las posiciones en las que el clasificador vale i y el atributo j vale k
                    y_k = []
                    for l in positionsOfk: #Por cada valor de posición de k, nos quedamos con las posiciones de y, formamos y_k
                        y_k.append(y[l])
                        
                    positionsOfk_i = np.where(y_k[0] == i) #Posiciones del valor de clasificación i en y
                    y_ki = []
                    for m in positionsOfk_i:
                        y_ki.append(y_k[0][m])
                        
                    #Nº de ejemplos clasificados como i en el atributo j con valor k    
                    n_i_j_k = len(y_ki[0])
                    
                    p_condics_j.append((n_i_j_k + self.k)/(countsClas[np.where(uniqueClas == i)][0] + self.k*len(valuesAtribj)))
                    
                p_condics.append(p_condics_j)
            
            list_p_condics.append(p_condics)
            self.flag_entrenado = True

    def clasifica_prob(self,ejemplo):
        if (self.flag_entrenado == False):
            raise ClasificadorNoEntrenado("Debe entrenar antes el clasificador.")
        else:
                
            #Por cada atributo de 'ejemplo', buscamos sus dos probabilidades condicionadas en la matriz construida
            probabilidades_c = {}
            
            for c in list(self.uniqueClas):
                posic_c = np.where(self.uniqueClas == c)[0][0]
                
                probs_condics = []
                
                for a in range(len(ejemplo)): #Por cada valor de los atributos, sacamos su probabilidad condicionada de la matriz construida
                    matriz_p_c = self.list_p_condics[posic_c] #Matriz para el valor de clasificación actual
                    valuesAtriba, numberOfValuesAtriba = np.unique(self.X[:,a], return_counts=True)
                    fila = a #Corresponde al nº de atributo
                    columna = np.where(valuesAtriba == ejemplo[a])[0][0] #Corresponde al valor del atributo
                    probs_condics.append(matriz_p_c[fila][columna])
                
                sum_logs = np.sum([math.log(x,10) for x in probs_condics])
                resultado = math.log(self.p_c[posic_c],10) + sum_logs
                probabilidades_c[c] = (resultado)
                probs_condics = []
                
            self.probabilidades_c = probabilidades_c
            return probabilidades_c

    def clasifica(self,ejemplo):
        if (self.flag_entrenado == False):
            raise ClasificadorNoEntrenado("Debe entrenar antes el clasificador.")
        else:
            maximo = {}
            find_max = max(self.clasifica_prob(ejemplo), key=self.clasifica_prob(ejemplo).get)
            maximo[find_max] = self.clasifica_prob(ejemplo)[find_max]
        
        return maximo

In [2]:
def rendimiento_p1(clasificador,X,y):
    y_prediccion = []
    for x in X:
        y_prediccion.append([k for k, v in clasificador.clasifica(x).items()][0])
    total=len(y)
    v_ok=[]
    for i in range(len(y)):
        if y[i]==y_prediccion[i]:
            v_ok.append(i)
    return len(v_ok)/len(y)

In [3]:
run "votos.py"

In [6]:
import math

In [4]:
X_votos_train, X_votos_test, y_votos_train, y_votos_test = train_test_split(datos, clasif, test_size=0.25)

In [8]:
nb_votos = NaiveBayes(k=0.5)
nb_votos.entrena(X_votos_train,y_votos_train)
rendimiento_votos=rendimiento_p1(nb_votos,X_votos_test,y_votos_test)
rendimiento_votos

0.8899082568807339

In [31]:
prueba = nb_votos.clasifica(X_votos_test[0]).get(1)
prueba

-4.902449346506935

In [42]:
from scipy.special import expit    


def suma_paralelo(a1, a2):
    """
        Recibe dos arrays y devuelve un array con las sumas de sus componentes sumadas
    """
    a3 = [a1[i]+a2[i] for i in range(len(a1))]
    return a3


def sigmoide(x):
    return expit(x)


def normaliza(X):
    """Normaliza los datos"""
    medias = np.mean(X, axis=0)
    desvs = np.std(X, axis=0)
    X_norm = (X - medias) / desvs
    return X_norm


class RegresionLogisticaMiniBatch():

    def __init__(self, clases=[0, 1], normalizacion=False,
                rate=0.1, rate_decay=False, batch_tam=64, n_epochs=200,
                 pesos_iniciales=None):
        
        mapa_reverse = {0: clases[0], 1: clases[1]}
        self.clases = clases
        self.mapa_reverse = mapa_reverse
        self.normalizacion = normalizacion
        self.rate = rate
        self.rate_decay = rate_decay
        self.batch_tam = batch_tam
        self.n_epochs = n_epochs
        self.pesos_iniciales = pesos_iniciales
        self.pesos = list()
        # si pesos está vacía, el clasificador no está entrenado
        self.flag_entrenado = False



    def entrena(self, X, y):
        
        pesos = []
        if self.pesos_iniciales is not None:  # tomamos los pesos directamente de la clase
            pesos = list(self.pesos_iniciales)
        else:  # iniciamos los pesos de forma aleatoria
            pesos = [random.random() for i in range(X.shape[1])]
        n_epochs = self.n_epochs
        y_2 = y.reshape(len(y), 1) #transformamos y para poder luego unirlo a X

        if self.normalizacion:
            X = normaliza(X)
                    
        # merge de los array para trabajar mejor con ellos
        big_chunk = np.concatenate((X, y_2), axis=1)
        # inicialización de parámetros
        batch_tam = self.batch_tam
        tasa_l = self.rate
        tasa_l0 = self.rate
        for i in range(n_epochs):
            chunks = np.array_split(big_chunk, batch_tam)
            # dividimos los datos en subconjuntos
            for block in chunks: #iteramos por cada minibatch
                # tomamos un subgrupo de datos
                # para cada subconjunto actualizamos
                pesos_previos = [0.0 for _ in block[0][:-1]]
                for array in block: #iteramos por cada ejemplo de nuestro minibatch para actualizar los pesos
                    sum_a = array[-1]-sigmoide(np.dot(pesos, array[:-1]))
                    sum_t = np.dot(sum_a, array[:-1])
                    pesos_previos = suma_paralelo(pesos_previos, sum_t)
                # una vez hecho todo el sumatorio de los elementos del subgrupo,
                # actualizamos los pesos reales multiplicando por la tasa de
                # aprendizaje y sumando
                
                act_b = np.dot(tasa_l, pesos_previos)
                pesos = suma_paralelo(pesos, act_b)
            if self.rate_decay:
                tasa_l = tasa_l0*(1/(1+i))
        self.pesos = pesos
        self.flag_entrenado = True

    def clasifica_prob(self, ejemplo):
        if self.flag_entrenado == False:
            raise ClasificadorNoEntrenado("Debe entrenar antes el clasificador.")
        else:
            result = sigmoide(np.dot(self.pesos, ejemplo))
            probs = dict()
            reverse= self.mapa_reverse
            probs[reverse.get(0)]=1-result
            probs[reverse.get(1)]=result
            return probs

    def clasifica(self, ejemplo):
        if self.flag_entrenado == False:
            raise ClasificadorNoEntrenado("Debe entrenar antes el clasificador.")
        else:
            #result = sigmoide(np.dot(self.pesos, ejemplo))
            #return self.mapa_reverse.get(round(result))
            maximo = {}
            find_max = max(self.clasifica_prob(ejemplo), key=self.clasifica_prob(ejemplo).get)
            maximo[find_max] = self.clasifica_prob(ejemplo)[find_max]
            
        return maximo



def rendimiento_p2(clasificador, X, y):
    """
    Devuelve el porcentaje de ejemplos bien clasificados
    :param clasificador: clasificador a emplear
    :param X: conjunto de ejemplos X
    :param y: clasificacion esperada
    """
    pred = []  # tablero que contenga nuestra prediccion para todos los ejemplos en X con el clasificador dado en argumento
    for i in range(len(X)):
        pred.append(clasificador.clasifica(X[i]))
    return len(np.where(pred == y)[0]) / len(X)

In [12]:
rendimiento_p2(nb_votos,X_votos_test,y_votos_test)

0.0

In [14]:
import random

In [43]:
lr_votos = RegresionLogisticaMiniBatch(rate=0.1,rate_decay=True,normalizacion=True)
lr_votos.entrena(X_votos_train, y_votos_train)

In [44]:
rendimiento_p1(lr_votos, normaliza(X_votos_test), y_votos_test)

0.944954128440367

In [47]:
rendimiento_p2(lr_votos, normaliza(X_votos_test), y_votos_test)

0.0

In [45]:
lr_votos.clasifica(normaliza(X_votos_test)[0])

{1: 0.9477529359854218}

In [46]:
nb_votos.clasifica(X_votos_test[0])

{1: -4.902449346506935}